# This section is bypassed as autoencoder models are worked so fine.

In [1]:
import warnings  

# Dimension Reduction 
from sklearn.decomposition import PCA 
from sklearn.manifold import TSNE
from umap import UMAP 

In [ ]:
def reduct_dimension(target_embeddings_matrix, label_embeddings_matrix, subject_dimension_reduction): 


    if subject_dimension_reduction[:3] == "PCA": # Eg. PCA_n2 
        n_components = int(subject_dimension_reduction.split("_n")[1]) 
        print(f"PCA: {n_components}") 
        pca = PCA(n_components) 
            
        # Target 
        target_embeddings_matrix = pca.fit_transform(target_embeddings_matrix) 
        print(f"PCA: {target_embeddings_matrix.shape}") 
        print(f"PCA: {pca.explained_variance_ratio_}") 
        print(f"PCA: {pca.explained_variance_ratio_.sum()}") 
        
        
        # Bypass Label 
        if label_embeddings_matrix == None:
            return target_embeddings_matrix, None 
        
        # Label 
        label_embeddings_matrix = pca.transform(label_embeddings_matrix) if label_embeddings_matrix is not None else [len(target_embeddings_matrix) * [0]] 
        
        return target_embeddings_matrix, label_embeddings_matrix  


    elif subject_dimension_reduction[:4] == "TSNE": # TSNE_nc2_p5_lr3 
        # Extract Components 
        n_components = int(subject_dimension_reduction.split("_nc")[1].split("_")[0])
        perplexity   = int(subject_dimension_reduction.split("_p")[1].split("_")[0])
        learning_rate = int(subject_dimension_reduction.split("_lr")[1].split("_")[0]) 
        print(f"TSNE: {n_components}, {perplexity}, {learning_rate}")

        # Preprocess: Matrix 
        if label_embeddings_matrix is not None:
            combined_matrix = np.vstack((target_embeddings_matrix, label_embeddings_matrix))
        else:
            combined_matrix = target_embeddings_matrix 

        # Reduction
        tsne = TSNE(n_components=n_components, perplexity=perplexity, learning_rate=learning_rate, random_state=42, init='random')
        # Target 
        transformed_combined_matrix = tsne.fit_transform(combined_matrix)

        # Split the transformed combined matrix back into target and label matrices
        if label_embeddings_matrix is not None:
            transformed_target_matrix = transformed_combined_matrix[:len(target_embeddings_matrix)]
            transformed_label_matrix = transformed_combined_matrix[len(target_embeddings_matrix):]
        else:
            transformed_target_matrix = transformed_combined_matrix
            transformed_label_matrix = None  # or an appropriate placeholder if needed

        # Target 
        target_embeddings_matrix = transformed_target_matrix 
        # Label 
        label_embeddings_matrix = transformed_label_matrix 

        return target_embeddings_matrix, label_embeddings_matrix  


    elif subject_dimension_reduction[:4] == "UMAP": # Eg. UMAP_nc3_nn60_min05

        with warnings.catch_warnings():
            # System 
            warnings.filterwarnings("ignore", message="n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.")
            # Extraction of components 
            n_components = int(subject_dimension_reduction.split("_nc")[1].split("_")[0]) 
            n_neighbors  = int(subject_dimension_reduction.split("_nn")[1].split("_")[0]) 
            min_dist     = str(subject_dimension_reduction.split("_min")[1]) 
            min_dist     = "0" + "." + min_dist[1] 
            min_dist     = float(min_dist) 
            print(f"UMAP: {n_components}, {n_neighbors}, {min_dist}") 

            # Target 
            umap_reducer  = UMAP(n_components = n_components, n_neighbors = n_neighbors, min_dist = min_dist, random_state = 42)
            target_embeddings_matrix = umap_reducer.fit_transform(target_embeddings_matrix) 
            print(f"UMAP: {target_embeddings_matrix.shape}")


            # Label 
            if label_embeddings_matrix is not None: 
                label_embeddings_matrix = umap_reducer.transform(label_embeddings_matrix) if label_embeddings_matrix is not None else [len(target_embeddings_matrix) * [0]]  
            else:
                label_embeddings_matrix = None 

        return target_embeddings_matrix, label_embeddings_matrix 
    
    else:
        print("No dimension reduction method selected")
        return target_embeddings_matrix, label_embeddings_matrix  